In [ ]:
# SIN NUMERO ALEATORIO

def extract_info( df ):
    
    options = Options()
    service = Service( ChromeDriverManager().install( ) )
    driver  = webdriver.Chrome( service = service )
    driver.maximize_window()

    url     = f'https://enlinea.sunedu.gob.pe/'
    driver.get( url )
    wait    = WebDriverWait( driver, 60 )
    
    verificar_titulos_popup = driver.find_element( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' )
    verificar_titulos_popup.click()
    
    frame_popup = driver.find_element( By.XPATH, '//*[@id="ifrmShowFormConstancias"]' )
    driver.switch_to.frame( frame_popup )
    
    for index, row in df.iterrows():
        
        dni_valor = row[ 'dni' ]
        
        dni_campo = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="doc"]' ) ) )
        dni_campo.clear()
        dni_campo.send_keys( dni_valor )
        
        max_retries = 5

        for retry in range( max_retries ):
        
            try: 

                time.sleep( 2 )
                captcha       = driver.find_element( By.XPATH, '//*[@id="captchaImg"]/img' )
                captcha_image = captcha.screenshot_as_png
                with open( 'image.png', 'wb' ) as f:
                    f.write( captcha_image )

                clave_valor   = pytesseract.image_to_string( Image.open( 'image.png' ),
                                                             config = '-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789' ).strip()
                campo_captcha = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="captcha"]' ) ) )
                campo_captcha.clear()
                campo_captcha.send_keys( clave_valor )                

                buscar_boton = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="buscar"]' ) ) )
                buscar_boton.click()
                
                print( f'Clave: { clave_valor }' )
                break
                
            except Exception as e:
                
                print( f'Error: { e }' )
                time.sleep( 5 )
                boton_x = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="closeModalError"]' ) ) )
                boton_x.click()
        
        grado         = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/b[1]/p' ) ) ).text
        fecha_diploma = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/b[2]' ) ) ).text
        modalidad     = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/span' ) ) ).text
        institucion   = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[3]/span' ) ) ).text
        
        print( f'Grado: { grado }' )
        print( f'Institutción: { institucion }' )
        
        df.at[ index, 'grado' ]         = grado       
        df.at[ index, 'fecha_diploma' ] = fecha_diploma    
        df.at[ index, 'modalidad' ]     = modalidad    
        df.at[ index, 'institucion' ]   = institucion
        
        # Salir del iframe para recargar la página principal
        driver.switch_to.default_content()

        # Recargar la página principal
        driver.refresh()

        # Volver a entrar en el iframe
        verificar_titulos_popup = driver.find_element( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' )
        verificar_titulos_popup.click()
        frame_popup = driver.find_element(By.XPATH, '//*[@id="ifrmShowFormConstancias"]')
        driver.switch_to.frame(frame_popup)

        time.sleep(10)
        print( '\n' )
        
    driver.quit()
        
    return df

In [ ]:
# CON FLUJO: ALGUNAS OBSERVACIONES MISSING

def scraper_SUNEDU( df ):
    
    options = Options()
    service = Service( ChromeDriverManager().install( ) )
    driver  = webdriver.Chrome( service = service )
    driver.maximize_window()

    url     = f'https://enlinea.sunedu.gob.pe/'
    driver.get( url )
    wait    = WebDriverWait( driver, 10 )
    
    verificar_titulos_popup = driver.find_element( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' )
    verificar_titulos_popup.click()
    
    frame_popup = driver.find_element( By.XPATH, '//*[@id="ifrmShowFormConstancias"]' )
    driver.switch_to.frame( frame_popup )
    
    variable_flag = False
    
    for index, row in df.iterrows():
        
        dni_valor = row[ 'dni' ]
        print( f'DNI: { dni_valor }' )
        
        dni_campo = WebDriverWait (driver, 60 ).until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="doc"]' ) ) )
        dni_campo.clear()
        dni_campo.send_keys( dni_valor )
        
        max_retries = 15

        for retry in range( max_retries ):
        
            try: 

                time.sleep( 2 )
                captcha       = driver.find_element( By.XPATH, '//*[@id="captchaImg"]/img' )
                captcha_image = captcha.screenshot_as_png
                
                ruta_img      = f'img/img.png'
                with open( ruta_img, 'wb' ) as f:
                    f.write( captcha_image )

                clave_valor   = pytesseract.image_to_string( Image.open( ruta_img ),
                                                             config = '-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789' ).strip()
                campo_captcha = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="captcha"]' ) ) )
                campo_captcha.clear()
                campo_captcha.send_keys( clave_valor )   

                buscar_boton = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="buscar"]' ) ) )
                buscar_boton.click()
                
                time.sleep( 10 )
                
                try: 
                    error_popup        = WebDriverWait (driver, 1 ).until( EC.visibility_of_element_located( ( By.XPATH, '//*[@id="frmError"]' ) ) )
                    class_error_popup  = error_popup.get_attribute( 'class' )
                    caso_no_resultados = 'No se encontraron resultados con este número de DNI'
                    error_msj          = driver.find_element( By.XPATH, '//*[@id="frmError_Body"]/p' ).text
                    
                    if error_msj.startswith( caso_no_resultados ):
                        
                        print( f'No se encontraron resultados' )
                        print( '\n' )   
                        
                        df.at[ index, 'grado' ]         = error_msj       
                        df.at[ index, 'fecha_diploma' ] = error_msj    
                        df.at[ index, 'modalidad' ]     = error_msj    
                        df.at[ index, 'institucion' ]   = error_msj
                        
                        driver.switch_to.default_content()
                        driver.refresh()
                        verificar_titulos_popup = driver.find_element( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' )
                        verificar_titulos_popup.click()
                        frame_popup = driver.find_element(By.XPATH, '//*[@id="ifrmShowFormConstancias"]')
                        driver.switch_to.frame(frame_popup)    
                        
                        variable_flag = True
                        
                        break
                
                except:   
                
                    grado         = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/b[1]/p' ) ) ).text
                    fecha_diploma = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/b[2]' ) ) ).text
                    modalidad     = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/span' ) ) ).text
                    institucion   = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[3]/span' ) ) ).text

                    ruta_img_nuev = os.path.join( os.path.dirname( ruta_img ), f'{ clave_valor }.png' )
                    os.rename( ruta_img, ruta_img_nuev )

                    print( f'Retry: { retry }' )
                    print( f'Clave: { clave_valor }' )
                    print( f'Grado: { grado }' )
                    print( f'Fecha del diploma: { fecha_diploma }' )
                    print( f'Modalidad: { modalidad }' )
                    print( f'Institutción: { institucion }' )
                    print( '\n' )    

                    df.at[ index, 'grado' ]         = grado       
                    df.at[ index, 'fecha_diploma' ] = fecha_diploma    
                    df.at[ index, 'modalidad' ]     = modalidad    
                    df.at[ index, 'institucion' ]   = institucion

                    driver.switch_to.default_content()
                    driver.refresh()
                    verificar_titulos_popup = driver.find_element( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' )
                    verificar_titulos_popup.click()
                    frame_popup = driver.find_element(By.XPATH, '//*[@id="ifrmShowFormConstancias"]')
                    driver.switch_to.frame(frame_popup)            

                    break
                    
                if variable_flag:
                    
                    break
                
            except Exception as e:
                
                # print( f'Error: { e }' )
                print( f'Error en la ejecución' )
                print( f'Intentando de nuevo después de 5 segundos...' )
                print( f'\n' )
                time.sleep( 5 )
                boton_x = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="closeModalError"]' ) ) )
                boton_x.click()
                
                continue
        
    driver.quit()
        
    return df

In [ ]:
# COMPLETO, PERO A VECES FALLA Y LE SOBRAN ALGUNAS COSAS

def scraper_SUNEDU( df ):
    
    options = Options()
    service = Service( ChromeDriverManager().install( ) )
    driver  = webdriver.Chrome( service = service )
    driver.maximize_window()

    url     = f'https://enlinea.sunedu.gob.pe/'
    driver.get( url )
    wait_60 = WebDriverWait( driver, 60 )
    wait_30 = WebDriverWait( driver, 30 )
    wait_10 = WebDriverWait( driver, 10 )
    wait_05 = WebDriverWait( driver, 5 )
    
    verificar_titulos_popup = wait_30.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' ) ) )
    verificar_titulos_popup.click()
    
    frame_popup = wait_30.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="ifrmShowFormConstancias"]' ) ) )
    driver.switch_to.frame( frame_popup )
    
    variable_flag = False
    
    df[ 'n_grados' ] = 0
    
    for index, row in df.iterrows():
        
        dni_valor = row[ 'dni' ]
        print( f'DNI: { dni_valor }' )
        
        dni_campo = wait_60.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="doc"]' ) ) )
        dni_campo.clear()
        dni_campo.send_keys( dni_valor )
        
        max_retries = 15

        for retry in range( max_retries ):
        
            try: 

                time.sleep( 2 )
                captcha       = wait_05.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="captchaImg"]/img' ) ) )
                captcha_image = captcha.screenshot_as_png
                
                ruta_img      = f'img/img.png'
                with open( ruta_img, 'wb' ) as f:
                    f.write( captcha_image )

                clave_valor   = pytesseract.image_to_string( Image.open( ruta_img ),
                                                             config = '-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789' ).strip()
                campo_captcha = wait_05.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="captcha"]' ) ) )
                campo_captcha.clear()
                campo_captcha.send_keys( clave_valor )   

                buscar_boton = wait_05.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="buscar"]' ) ) )
                buscar_boton.click()
                
                time.sleep( 10 )
                
                try: 
                    
#                     grado         = wait_05.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/b[1]/p' ) ) ).text
#                     fecha_diploma = wait_05.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/b[2]' ) ) ).text
#                     modalidad     = wait_05.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/span' ) ) ).text
#                     institucion   = wait_05.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[3]/span' ) ) ).text

#                     ruta_img_nuev = os.path.join( os.path.dirname( ruta_img ), f'{ clave_valor }.png' )
#                     os.rename( ruta_img, ruta_img_nuev )

#                     print( f'Retry: { retry }' )
#                     print( f'Clave: { clave_valor }' )
#                     print( f'Grado: { grado }' )
#                     print( f'Fecha del diploma: { fecha_diploma }' )
#                     print( f'Modalidad: { modalidad }' )
#                     print( f'Institutción: { institucion }' )
#                     print( '\n' )    

#                     df.at[ index, 'grado' ]         = grado       
#                     df.at[ index, 'fecha_diploma' ] = fecha_diploma    
#                     df.at[ index, 'modalidad' ]     = modalidad    
#                     df.at[ index, 'institucion' ]   = institucion
                    sections        = {}
                    result_sections = wait.until( EC.presence_of_all_elements_located( (By.XPATH, '//*[@id="finalData"]/tr') ) )
                    n_grados        = len( result_sections )

                    for i, result in enumerate( result_sections ):
                        boxes = wait.until( EC.presence_of_all_elements_located( ( By.XPATH, f'//*[@id="finalData"]/tr[{ i + 1 }]/td' ) ) )
                        for j, box in enumerate( boxes ):
                            boxes_names_list = { 0: 'datos', 1: 'grado_fecha', 2: 'institucion', 3: 'otro' }
                            box_name = boxes_names_list.get( j )   
                            text     = box.text
                            sections[ f'{ box_name }_{ i }' ] = text

                    row = { 'dni': dni_valor }
                    row.update(sections)
                    df.loc[index, ['dni', *sections.keys() ] ] = row.values()
                    df.loc[ index, [ 'n_grados' ] ] = n_grados

                    
                    print( 'Exito' )
                    
                    # ------------------------

                    driver.switch_to.default_content()
                    driver.refresh()
                    
                    verificar_titulos_popup = wait_30.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' ) ) )
                    verificar_titulos_popup.click()
                    
                    frame_popup = wait_30.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="ifrmShowFormConstancias"]') ) )
                    driver.switch_to.frame( frame_popup )    
                    
                    print( 'pasamos a la siguiente observacion' )

                    break                    
                
                except:   
                
                    error_popup        = wait_05.until( EC.visibility_of_element_located( ( By.XPATH, '//*[@id="frmError"]' ) ) )
                    class_error_popup  = error_popup.get_attribute( 'class' )
                    caso_no_resultados = 'No se encontraron resultados con este número de DNI'
                    error_msj          = wait_05.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="frmError_Body"]/p' ) ) ).text
                    
                    if error_msj.startswith( caso_no_resultados ) == False:
                        
                        raise Exception( 'Continuar hacia el bloque except' )
                    
                    else:
                        
                        print( f'No se encontraron resultados' )
                        print( '\n' )   
                        
                        # df.at[ index, 'grado' ]         = error_msj       
                        # df.at[ index, 'fecha_diploma' ] = error_msj    
                        # df.at[ index, 'modalidad' ]     = error_msj    
                        # df.at[ index, 'institucion' ]   = error_msj
                        
                        columnas_a_actualizar       = df.columns.difference( [ 'dni', 'n_grados' ] )
                        df[ columnas_a_actualizar ] = error_msj
                        
                        driver.switch_to.default_content()
                        driver.refresh()
                        
                        verificar_titulos_popup = wait_30.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' ) ) )
                        verificar_titulos_popup.click()
                        
                        frame_popup = wait_30.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="ifrmShowFormConstancias"]' ) ) )
                        driver.switch_to.frame( frame_popup )    
                        
                        variable_flag = True                        
                        
                        break
                    
                if variable_flag:
                    
                    break
                
            except Exception as e:
                
                # print( f'Error: { e }' )
                print( f'Error en la ejecución' )
                print( f'Intentando de nuevo después de 5 segundos...' )
                print( f'\n' )
                boton_x = wait_30.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="closeModalError"]' ) ) )
                boton_x.click()
                
                continue
        
    driver.quit()
        
    return df